In [1]:
import numpy as np
import yfinance as yf

In [2]:
# Download S&P 500 front month futures data
prices = yf.download('ES=F')['Close']
prices 

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Ticker,ES=F
Date,
2000-09-18,1467.50
2000-09-19,1478.50
2000-09-20,1469.50
2000-09-21,1469.50
2000-09-22,1468.50
...,...
2025-02-18,6146.75
2025-02-19,6163.00
2025-02-20,6136.50


## Logarithmic returns 

$$ r_{t,t+1} = (P_{t+1}- P_{t})/P_{t} $$ 

or 
$$ 
r_{t,t+1} = P_{t+1}/P_{t} -1
$$ 

## Log returns 
$$
log(r_{t,t+1} ) = log(P_{t+1}) - log(P_{t}) - log(1)
$$

$$
= log(P_{t+1}) - log(P_{t}) 
$$

In [5]:
# Calculate daily logarithmic returns
log_returns = np.log(prices).diff()
log_returns.dropna(inplace = True )
log_returns

Ticker,ES=F
Date,
2000-09-19,0.007468
2000-09-20,-0.006106
2000-09-21,0.000000
2000-09-22,-0.000681
2000-09-25,-0.005120
...,...
2025-02-18,0.000122
2025-02-19,0.002640
2025-02-20,-0.004309


In [ ]:
# Annualize returns and volatility
# 1 year has 252 trading days
periods_per_year = 252
# count the number of days available in our data
n_periods = log_returns.shape[0]
# already in 1+return format

annualized_return = np.exp(log_returns.sum()) **(periods_per_year/n_periods)  -1 
annualized_return = annualized_return.values[0]
annualized_vol = (np.exp(log_returns)-1) .std() * np.sqrt(252)
annualized_vol = annualized_vol.values[0]

print(f"Annualized return: {annualized_return:.4f}")
print(f"Annualized volatility: {annualized_vol:.4f}")

Annualized return: 0.0593
Annualized volatility: 0.1940


In [9]:
np.exp(log_returns)

Ticker,ES=F
Date,
2000-09-19,1.007496
2000-09-20,0.993913
2000-09-21,1.000000
2000-09-22,0.999319
2000-09-25,0.994893
...,...
2025-02-18,1.000122
2025-02-19,1.002644
2025-02-20,0.995700


## Get the Risk Free Rate 

In [16]:
# Download the 3-month Treasury bill rate as the risk-free rate
sp500_start_date = str(prices.index[0])[:10]
print("sp500_start_date: ", sp500_start_date)
risk_free_rates = yf.download('^IRX', start=sp500_start_date)['Close']
avg_risk_free_rate = risk_free_rates.mean() / 100
avg_risk_free_rate = avg_risk_free_rate.values[0]

sp500_start_date:  2000-09-18


[*********************100%***********************]  1 of 1 completed


In [17]:
avg_risk_free_rate

np.float64(0.016848480195177366)

In [20]:
# Calculate the Sharpe Ratio
sharpe_ratio = (annualized_return - avg_risk_free_rate) / annualized_vol
# Calculate the Sortino Ratio
downside_vol = log_returns[log_returns<0].std() * np.sqrt(252)
downside_vol = downside_vol.values[0]
sortino_ratio = (annualized_return - avg_risk_free_rate) / downside_vol

# Calculate the Calmar Ratio
# create wealth index
cum_returns = np.exp(log_returns.cumsum())
drawdowns = (cum_returns.cummax() - cum_returns) / cum_returns.cummax()
max_drawdown = np.max(drawdowns)
calmar_ratio = annualized_return / max_drawdown

In [21]:
print()
print(f"annualized_return: {np.round(annualized_return * 100, 1)}%")
print(f"avg_risk_free_rate: {np.round(avg_risk_free_rate * 100, 1)}%")
print(f"annualized_volatility: {np.round(annualized_vol * 100, 1)}%")
print(f"downside_volatility: {np.round(downside_vol * 100, 1)}%")
print(f"max_drawdown: {np.round(max_drawdown * 100, 1)}%")
print()
print(f"sharpe_ratio: {np.round(sharpe_ratio, 2)}")
print(f"sortino_ratio: {np.round(sortino_ratio, 2)}")
print(f"calmar_ratio: {np.round(calmar_ratio, 2)}")
print()


annualized_return: 5.9%
avg_risk_free_rate: 1.7%
annualized_volatility: 19.4%
downside_volatility: 15.9%
max_drawdown: 57.1%

sharpe_ratio: 0.22
sortino_ratio: 0.27
calmar_ratio: 0.1

